In [1]:
import jsonlines
from dialop.games import OptimizationGame
from dialop.games.optimization import WORKERS, TASKS

with jsonlines.open('data/optimization.jsonl') as jsonl_f:
     data = [obj for obj in jsonl_f]


In [11]:
import random
idx = random.randint(0, len(data))

def stringify_proposal(assignment):
    assignment.sort(key=lambda x: x[1])
    out_str = ""
    out_str = '\n'.join([f"{TASKS[a[1]]}: {WORKERS[a[0]]} ({game.table.values[a[0]][a[1]]})" for a in assignment])
    return out_str

d = data[idx]
print(f"Dialog idx: {idx}")

game = OptimizationGame.create_from_game_state(d)

table = game.tables[0]
obs_string = "Player 1 knowledge: \n" 
for row in table[1:]:
    obs_string += row[0] + ": "
    obs_string += ', '.join([f"{c} for {TASKS[i]}" for i, c in enumerate(row[1:]) if type(c) == int])
    obs_string += " \n "
print(obs_string)

table = game.tables[1]
obs_string = "Player 2 knowledge: \n" 
for row in table[1:]:
    obs_string += row[0] + ": "
    obs_string += ', '.join([f"{TASKS[i]}:{c}" for i, c in enumerate(row[1:]) if type(c) == int])
    obs_string += " \n "
print(obs_string)

print(f"{'-'*50} CONVERSATION {'-'*50}")
final_proposal = None
for a in d['action_log']:
    output_str = f"Player {a['player']+1}: "
    if a['type'] == 'message':
        output_str += a['message']['data']
    elif a['type'] == 'proposal':
        output_str += a['proposal'].replace('<br/>&emsp;', '\n')
        final_proposal = a['proposal_ids']
    elif a['type'] == 'proposal_response':
        output_str += 'ACCEPT' if a['response']['accept'] else 'REJECT'
    print(output_str)

print("-"*100)
best_assignment, best_reward = game.table.find_max_value_known_assignment([d['mask1'], d['mask2']])
print("\nFINAL PROPOSAL:")
print(stringify_proposal(final_proposal))
print(f"Proposal reward = {d['proposal_reward']}")
print("\nOPTIMAL ASSIGNMENT:")
print(stringify_proposal(best_assignment))
print(f"Best possible reward = {d['best_assignment_reward']}")

Dialog idx: 64
Player 1 knowledge: 
Ava Li: 19 for BLEU, 311 for QuAC, 192 for SWAG 
 Daniel Nguyen: 162 for Electra 
 Sofia Patel: 142 for BLEU, 119 for Electra, 251 for LLaMA 
 Andrei Petrov: 0 for BLEU, 152 for LLaMA 
 Morgan Reed: 76 for BLEU, 178 for LLaMA, 119 for QuAC 
 Joseph Santos: 284 for BLEU, 122 for GLUE, 241 for LLaMA, 281 for RoBERTa, 69 for SWAG 
 Ethan Smith: 241 for BLEU, 324 for GLUE, 62 for RoBERTa, 115 for QuAC 
 Noah Wilson: 135 for LLaMA, 158 for SWAG 
 
Player 2 knowledge: 
Ava Li: BLEU:49, Electra:604, LLaMA:57, RoBERTa:662, QuAC:778 
 Daniel Nguyen: Electra:405, GLUE:297, RoBERTa:132 
 Sofia Patel: GloVe:811, GLUE:140, LLaMA:629, QuAC:413 
 Andrei Petrov: BLEU:0, GloVe:99, GLUE:389, QuAC:107 
 Morgan Reed: Electra:587, GloVe:8, LLaMA:446, SWAG:231 
 Joseph Santos: QuAC:256 
 Ethan Smith: Electra:364, GloVe:372, LLaMA:240 
 Noah Wilson: SWAG:397 
 
-------------------------------------------------- CONVERSATION -------------------------------------------------